# Magic Facies Prediction

In [162]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.pipeline import make_pipeline
import sys
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from plotly import graph_objs as go
from plotly import tools
# Import our custom modules
sys.path.append("..")
from mysticbit import ml, munging, plots

## Data Wrangling

In [66]:
df = munging.load_log_data()
df.head()

,PSEUDO_DEPTH,TEMP,CALI,GR,ILD,NPHI,RHOB,DT,TVD,TVDSS,...,PHIE,PHIT,SW,EF,PERFOS,RES_ID,WELL_ID,X,Y,HACKANAME
0,2074.53,87.19,10.31,55.08,3.3128,0.1678,2.3124,108.95,1588.62,1566.62,...,0.0591,0.2066,0.5863,5,0,3005,210075859,15,20.0,B03
1,2074.68,87.20,10.25,52.89,3.6592,0.1739,2.2895,110.91,1588.77,1566.77,...,0.0871,0.2222,0.5225,5,0,3005,210075859,15,20.0,B03
2,2074.83,87.20,10.20,52.54,4.1220,0.1618,2.2686,114.92,1588.92,1566.92,...,0.0730,0.2061,0.5004,5,0,3005,210075859,15,20.0,B03
3,2074.98,87.21,10.18,54.36,4.7189,0.1448,2.2533,119.66,1589.06,1567.06,...,0.0335,0.1768,0.4566,5,0,3005,210075859,15,20.0,B03
4,2075.90,87.24,9.76,48.13,13.0483,0.1085,2.0885,126.26,1589.94,1567.94,...,0.0331,0.1412,0.2211,1,1,3005,210075859,15,20.0,B03


In [67]:
print(f"Titles: {' '.join(list(df))}")

Titles: PSEUDO_DEPTH TEMP CALI GR ILD NPHI RHOB DT TVD TVDSS VCL PHIE PHIT SW EF PERFOS RES_ID WELL_ID X Y HACKANAME


In [79]:
df.shape

(12306, 23)

## Facies Synthetics

As we don't have any facies data to form our training dataset we need to generate some from the well logs we do have:
- GR - Gamma Ray
- RHOB - Density
- NPHI - Neutron Porosity
- DT - Sonic Log

Then split it into facies using K-Means clustering

In [72]:
df['FACIES'] = ml.create_facies(df, 5)

In [73]:
log = df[df['HACKANAME'] == 'B03']

In [74]:
log = log[['TVDSS', 'GR', 'RHOB', 'NPHI', 'DT', 'FACIES']]

In [77]:
gamma_ray = go.Scatter(
    x = log['GR'],
    y = log['TVDSS'],
    name = 'GR'
)

density = go.Scatter(
    x = log['RHOB'],
    y = log['TVDSS'],
    name = 'RHOB'
)

neutron_density = go.Scatter(
    x = log['NPHI'],
    y = log['TVDSS'],
    name = 'NPHI'
)

sonic_log = go.Scatter(
    x = log['DT'],
    y = log['TVDSS'],
    name = 'DT'
)

facies_estimation = go.Scatter(
    x = log['FACIES'],
    y = log['TVDSS'],
    name = 'Facies'
)

layout = go.Layout(
    autosize=False,
    width=200,
    height=1000
)

yaxis=dict(zeroline=False, showline=False)

fig = tools.make_subplots(rows=1, cols=5)

fig.append_trace(gamma_ray, 1, 1)
fig.append_trace(density, 1, 2)
fig.append_trace(neutron_density, 1, 3)
fig.append_trace(sonic_log, 1, 4)
fig.append_trace(facies_estimation, 1, 5)

fig['layout'].update(height=1000, width=950)
fig['layout'].update( title='Facies QC Curves')

iplot(fig, filename='basic-line')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]  [ (1,4) x4,y4 ]  [ (1,5) x5,y5 ]



## Facies Identification Model

Train model to predict synthetic facies type from GR, RHOB, NPHI and DT curves

In [144]:
xs = df[['GR', 'RHOB', 'NPHI', 'DT']]
xs.head(2)

,GR,RHOB,NPHI,DT
0,55.08,2.3124,0.1678,108.95
1,52.89,2.2895,0.1739,110.91


In [145]:
ys = df[['FACIES']]
ys.head(2)

,FACIES
0,2
1,2


In [136]:
fcs_idnt_mdl = LogisticRegression().fit(xs, ys)

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



Very simple test against some dummy data for a very quick model qc before moving on to more through validation

In [142]:
test = pd.DataFrame()
test['GR'] = [30]
test['RHOB'] = [2]
test['NPHI'] = [0.1]
test['DT'] = [140]
test

,GR,RHOB,NPHI,DT
0,30,2,0.1,140


In [143]:
pre_test = fcs_idnt_mdl.predict(test)
print(pre_test)

[0]


Lets try finding model accuaracy by using leave one well out cross validation (we are hoping to use this on blind wells after all!)

In [169]:
def lowocv(xs, ys, wells, df, model):
    acc = []
    for well in wells:
        training_indicies = df.index[df['HACKANAME'] != well].tolist()
        test_indicies = df.index[df['HACKANAME'] == well].tolist()
        train_x = xs.loc[training_indicies]
        train_y = ys.loc[training_indicies]
        temp_model = model.fit(train_x, train_y)
        test_x = xs.loc[test_indicies]
        test_y = ys.loc[test_indicies]
        pred_y = temp_model.predict(test_x)
        acc.append(accuracy_score(test_y, pred_y))
    return acc

In [191]:
def proba(xs, ys, wells, df, model):
    probs = []
    for well in wells:
        training_indicies = df.index[df['HACKANAME'] != well].tolist()
        test_indicies = df.index[df['HACKANAME'] == well].tolist()
        train_x = xs.loc[training_indicies]
        train_y = ys.loc[training_indicies]
        temp_model = model.fit(train_x, train_y)
        test_x = xs.loc[test_indicies]
        test_y = ys.loc[test_indicies]
        probs.append(temp_model.predict_proba(test_x))
    return probs

In [149]:
wells = df['HACKANAME'].unique()

In [150]:
logr_mdl = LogisticRegression()

In [157]:
facies_id_acc = lowocv(xs, ys['FACIES'], wells, df, logr_mdl)

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: 

In [158]:
sum(facies_id_acc) / len(facies_id_acc)

0.8580804912867569

So keeping things simple, let's see if we can improve our prediction using polynomial logistic regression

In [182]:
poly_model = make_pipeline(PolynomialFeatures(3), LogisticRegression())

In [183]:
facies_id_acc = lowocv(xs, ys['FACIES'], wells, df, poly_model)

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: 

In [184]:
sum(facies_id_acc) / len(facies_id_acc)

0.9108305360867577

In [185]:
facies_id_acc

[0.9308755760368663,
 0.9076819407008087,
 0.921875,
 0.9144542772861357,
 0.9223454833597464,
 0.9234828496042217,
 0.73125,
 0.8720602069614299,
 0.96579476861167,
 0.9142857142857143,
 0.9681440443213296,
 0.9319371727748691,
 0.9503619441571872,
 0.8990825688073395,
 0.8735042735042735,
 0.888135593220339,
 0.95822454308094,
 0.9214536928487691]

In [192]:
test1 = proba(xs, ys['FACIES'], wells, df, poly_model)

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: 

In [193]:
test1

[array([[9.53807793e-04, 3.52804827e-06, 9.18575716e-01, 1.13953977e-03,
         7.93274081e-02],
        [4.87473128e-03, 2.21749327e-06, 9.68586703e-01, 5.00306708e-05,
         2.64863175e-02],
        [2.68861408e-02, 9.64634168e-08, 9.67666873e-01, 4.46770129e-06,
         5.44242203e-03],
        ...,
        [1.48030306e-05, 1.02579814e-04, 3.70150788e-02, 4.99709249e-01,
         4.63158290e-01],
        [7.45774441e-06, 6.27011362e-05, 8.64110512e-02, 4.82932392e-01,
         4.30586398e-01],
        [3.50538222e-06, 2.03261099e-05, 6.53957201e-02, 4.83396606e-01,
         4.51183843e-01]]),
 array([[6.12323320e-07, 4.15416637e-06, 6.53354395e-03, 4.96109100e-01,
         4.97352589e-01],
        [1.70067896e-06, 2.13333704e-05, 1.81267032e-02, 4.94121857e-01,
         4.87728405e-01],
        [3.28020316e-06, 6.34033475e-05, 3.49902929e-02, 4.88186096e-01,
         4.76756928e-01],
        ...,
        [1.07791209e-01, 1.01381531e-12, 8.92208705e-01, 2.72031042e-08,
        

## Calculate Lag Data

In [98]:
gr_df = munging.create_ml_dataframe(df, feature_cols=['GR'], label_cols=['GR'])

In [99]:
print(f"Titles: {' '.join(list(ml_df))}")

Titles: HACKANAME TVDSS GR RES_ID GR_lag_0 GR_lag_2 GR_lag_4 GR_lag_6 GR_lag_8 GR_lag_10 GR_lag_12 GR_lag_14 GR_lag_16 GR_lag_18 GR_lag_20 GR_lag_22 GR_lag_24 GR_lag_26 GR_lag_28 GR_lag_30 GR_lag_32 GR_lag_34 GR_lag_36 GR_lag_38 GR_lag_40 GR_lag_42 GR_lag_44 GR_lag_46 GR_lag_48 GR_futr_2 GR_futr_4 GR_futr_6 GR_futr_8 GR_futr_10


In [100]:
gr_df.shape

(999, 34)

In [101]:
rhob_df = munging.create_ml_dataframe(df, feature_cols=['RHOB'], label_cols=['RHOB'])

In [102]:
rhob_df.shape

(999, 34)

In [103]:
nphi_df = munging.create_ml_dataframe(df, feature_cols=['NPHI'], label_cols=['NPHI'])

In [104]:
nphi_df.shape

(999, 34)

In [105]:
dt_df = munging.create_ml_dataframe(df, feature_cols=['DT'], label_cols=['DT'])

In [106]:
dt_df.shape

(999, 34)

In [107]:
facies_df = munging.create_ml_dataframe(df, feature_cols=['FACIES'], label_cols=['FACIES'])

In [108]:
facies_df.shape

(999, 34)

## Predicting Facies Using Subsurfae Locaton and Predicted Gamma

In [206]:
x2 = df[['GR', 'RHOB', 'NPHI', 'DT']]
x2.head(2)

,GR,RHOB,NPHI,DT
0,55.08,2.3124,0.1678,108.95
1,52.89,2.2895,0.1739,110.91


In [209]:
# Feature Importance
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(x2, ys)
# display the relative importance of each attribute
print(model.feature_importances_)

d:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

d:\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



[0.20563787 0.2240205  0.30146421 0.26887741]


In [207]:
logr_mdl = LogisticRegression()

In [208]:
sp = lowocv(x2, ys['FACIES'], wells, df, logr_mdl)

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: 

In [202]:
sum(sp) / len(sp)

0.45209386049820843

In [203]:
poly_model = make_pipeline(PolynomialFeatures(3), LogisticRegression())

In [204]:
sp = lowocv(x2, ys['FACIES'], wells, df, logr_mdl)

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

d:\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: 

In [205]:
sum(sp) / len(sp)

0.45209386049820843